# Creación de un Docker
by [Cristian Rodríguez](https://github.com/cfrc2694)

En el enfoque actual, se tienen dos tipos de usuarios:

 - El administrador del docker es el encargado de crea paracada usuario final, un contenedor basado en la imagen preparada por el profesor Joel Jones de la PUCP, https://hub.docker.com/r/jjonesperez/pucp-madgraph-pythia-delphes, ya instalada en su versión 0.7 en el hep-server2, y modificada en la imagen local `pheno_collider_tools` para incluir un servidor de jupyterlab. **CUIDADO: el administrador del docker tiene permisos sobre la totalidad del sistema de ficheros de hep-server2, por lo tanto debe cuidar de no heredar los permisos a cada usuario final, esto se aclara más adelante.**

 - El usuario final es el encargado de conectar a su contenedor via ssh donde un servidor jupyter-lab estará corriendo, y podrá ejecutar los notebooks que se encuentren en su directorio de trabajo. En caso de requerirlo, el usuario deberá preguntar al administrador del docker para reiniciar el contenedor.


## Conexión ssh al servidor de fenomenología: hep-server2
Desde una terminal bash, powershell o cmd, se debe ejecutar el siguiente comando, donde se reemplaza `docker_admin_name` el nombre del administrador del docker

```bash
ssh -Y docker_admin_name@hep-server2.uniandes.edu.co 
```

## Puertos en uso
Cada usuario final se conectará a su contenedor vía un *port-forwarding*, para cada usuario nuevo se debe tener en cuenta que cada contenedor debe tener un puerto diferente, por lo tanto se debe tener un registro de los puertos empleados.

Los siguientes puertos están reservados para el sistema:
- 22: puerto de conexión ssh al servidor hep-server2 (expuesto a internet)
- 2222: puerto de conexión ssh al servidor hep-server2 (expuesto a la red interna de la universidad)
- 80: puerto de conexión http al servidor hep-server2 (expuesto a la red interna de la universidad)
- 443: puerto de conexión https al servidor hep-server2 (expuesto a la red interna de la universidad)

Para consultar los puertos en uso se debe ejecutar el siguiente comando en el servidor hep-server2:

In [1]:
docker ps

CONTAINER ID   IMAGE                         COMMAND       CREATED      STATUS      PORTS                                                 NAMES
8091c1c13691   pheno_collider_tools:latest   "/bin/bash"   8 days ago   Up 8 days   0.0.0.0:1401->1401/tcp, :::1401->1401/tcp, 2694/tcp   jupyroot_gardila
cfffc1d7e28a   pheno_collider_tools:latest   "/bin/bash"   8 days ago   Up 8 days   2694/tcp, 0.0.0.0:4950->4950/tcp, :::4950->4950/tcp   jupyroot_j.penuela
b4ec71afccca   pheno_collider_tools:latest   "/bin/bash"   8 days ago   Up 8 days   0.0.0.0:2694->2694/tcp, :::2694->2694/tcp             jupyroot_c.rodriguez45



Note que en este ejemplo, los puertos 1401, 2502, y 4950 están en uso. 

Al usuario nuevo por (ejemplo `c.rodriguez45`) se le asignará un puerto, preferíblemente de 4 dígitos (por ejemplo `1234`), y se le asignará un directorio de trabajo en el `/disco4/personal_folders`, por favor guardar en las siguientes variables el puerto y el nombre de usuario final:

In [2]:
# declare variables port, user_name, and password.
port=1234
user_name=c.rodriguez45
password=random-pass-here

Verifica que la información es correcta:

In [3]:
personal_folder=/disco4/personal_folders/$user_name
echo "Al usuario $user_name"
echo "se le asigna el puerto $port"
echo "en la carpeta $personal_folder"

Al usuario c.rodriguez45
se le asigna el puerto 1234
en la carpeta /disco4/personal_folders/c.rodriguez45



## Creación de un contenedor para un usuario final

En el servidor hep-server2, se tienen instalados 4 discos de alta capacidad `/disco1`, `/disco2`, `/disco3`, y `/disco4` destinados a almacenar las simulaciones, el disco 4 tiene destinado un espacio para los archivos de prueba, notebooks y demás que puedan ser compartidos entre los usuarios. El acceso a estos discos será de sólo lectura para los usuarios finales.

Los usuarios finales tendrán un directorio de trabajo en el disco 4, en la ruta `/disco4/personal_folders/usuario_final`, donde `usuario_final` es el nombre del usuario final. La carpeta personal de cada usuario final específico, estará montada en el contenedor respectivo en la ruta `/home`. Donde a su vez estará montado el servidor de jupyter-lab.

bajo los dos siguientes comandos se crea la carpeta del usuario final y se crea el contenedor de docker donde se evidencia los permisos de sólo lectura sobre los discos para el usuario final, se asigna el puerto especificado en la variable `port`, y se monta el directorio de trabajo del usuario final en el contenedor en la ruta `/home`. 


```bash
mkdir -p $personal_folder
docker run -it -t \
    --mount type=bind,source=/disco1,target=/disco1,readonly \
    --mount type=bind,source=/disco2,target=/disco2,readonly \
    --mount type=bind,source=/disco3,target=/disco3,readonly \
    --mount type=bind,source=/disco4,target=/disco4 \
    --mount type=bind,source=/Madgraph_Simulations,target=/Madgraph_Simulations,readonly \
    --mount type=bind,source=$personal_folder,target=/home/pheno \
    --device /dev/fuse -d -p $port:$port \
    --name jupyroot_$user_name \
    pheno_collider_tools:latest;
```	

Puedes consultar que efectivamente se crea el contenedor, en el puerto adecuado:

In [1]:
docker ps -a

CONTAINER ID   IMAGE                         COMMAND       CREATED        STATUS        PORTS                                                 NAMES
8091c1c13691   pheno_collider_tools:latest   "/bin/bash"   12 hours ago   Up 12 hours   0.0.0.0:1401->1401/tcp, :::1401->1401/tcp, 2694/tcp   jupyroot_gardila
cfffc1d7e28a   pheno_collider_tools:latest   "/bin/bash"   12 hours ago   Up 12 hours   2694/tcp, 0.0.0.0:4950->4950/tcp, :::4950->4950/tcp   jupyroot_j.penuela
b4ec71afccca   pheno_collider_tools:latest   "/bin/bash"   12 hours ago   Up 12 hours   0.0.0.0:2694->2694/tcp, :::2694->2694/tcp             jupyroot_c.rodriguez45



Ya casi está todo listo, ahora se debe configurar el contenedor para que el usuario final pueda conectarse vía ssh, y pueda ejecutar el servidor de jupyter-lab:

In [5]:
# Primero iniciamos el contenedor
docker start jupyroot_$user_name

jupyroot_c.rodriguez45



La siguiente serie de comandos añade las siguientes funciones al bash del contendor para resolver los problemas de permisos

- `print_ids` imprime los ids del usuario final.
- `get_userid` obtiene el id del usuario final.
- `get_groupid` obtiene el id del grupo del usuario final.
- `change_permissions` cambia los permisos de los archivos y carpetas del usuario final para que pueda escribir en su directorio de trabajo, cuando no se conecta al contenedor sino que se conecta al servidor hep-server2.

Adicionalmente se ejecuta el servidor de jupyter-lab en el puerto asignado al usuario final. La contraseña por defecto es el nombre del usuario final, por ejemplo `c.rodriguez45`. Se le recomienda al usuario final cambiar la contraseña por defecto.

```bash
# Añade una función que imprima el userid y el groupid del usuario
userid=$(id -u $user_name)
groupid=$(id -g $user_name)
# En el servidor hay un usuario llamado pheno con ID 1000, se debe cambiar el ID de pheno al ID del usuario final, para que el usuario final pueda escribir en su directorio de trabajo
docker exec -it jupyroot_$user_name chown -R $userid:$groupid /home/pheno 
# cambia el id del usuario en /etc/passwd
docker exec -it jupyroot_$user_name sed -i 's/1000/'$userid'/g' /etc/passwd
# cambia el id del grupo en /etc/group
docker exec -it jupyroot_$user_name sed -i 's/1000/'$groupid'/g' /etc/group
# add user to wheel group
docker exec -it jupyroot_$user_name usermod -aG wheel pheno
# cambia la contraseña del pheno
docker exec -it jupyroot_$user_name bash -c "echo pheno:$password | chpasswd"
# cambia el id del grupo en /etc/shadow
docker exec -it jupyroot_$user_name sed -i 's/1000/'$groupid'/g' /etc/shadow
# extrae /home/pheno.tar.gz 
docker exec -u $userid -it jupyroot_$user_name bash -c 'cd / ; tar -xvf /home/pheno.tar.gz'
# add source /Collider/ROOT/installROOT/bin/thisroot.sh to /etc/bashrc
docker exec -it jupyroot_$user_name bash -c 'echo "source /Collider/ROOT/installROOT/bin/thisroot.sh" >> /etc/bashrc'
# add a file jupyter_run.sh (delete it if exists) to /home/pheno to execute at the start of the container with the user pheno
docker exec -u $userid -it jupyroot_$user_name bash -c 'echo "export PS1=\"\[\e[32m\][\u:\[\e[34m\]\w\[\e[32m\]]\$\[\e[0m\] \"; cd ~ ; source /Collider/env/bin/activate; jupyter-lab --ip 0.0.0.0 --port '$port' --no-browser --ServerApp.token='$password'" > /home/pheno/jupyter_run.sh'
# configure container to run jupyter_run.sh as user pheno at the start of the container
docker exec -it jupyroot_$user_name bash -c "echo 'su pheno -c \"bash /home/pheno/jupyter_run.sh\"' >> /root/.bashrc"
# dar permisos de ejecución a jupyter_run.sh
docker exec -it jupyroot_$user_name bash -c 'chmod +x /home/pheno/jupyter_run.sh'
# reiniciar el contenedor
docker restart jupyroot_$user_name
```

Y Listo! el usuario final puede conectarse al servidor de jupyter-lab en el puerto asignado, y ejecutar los notebooks que se encuentren en su directorio de trabajo.


## Conexión al servidor de jupyter-lab
Para la conexión al servidor de jupyter-lab, se debe ejecutar el siguiente comando en una terminal bash, powershell o cmd en el escritorio del usuario final, donde se reemplaza `user` por el nombre del usuario final, y `port` por el puerto asignado al usuario final:

```bash
    ssh user@hep-server2.uniandes.edu.co -NL port:localhost:port
```
donde `user` se podrá conectar a su jupyter-lab en la siguiente dirección: http://localhost:port 

Por ejemplo, para el usuario `c.rodriguez45` con el puerto `1234` se debe ejecutar el siguiente comando:
```bash
    ssh c.rodriguez45@hep-server2.uniandes.edu.co -NL 1234:localhost:1234
```
donde c.rodriguez45 se podrá conectar a su jupyter-lab en la siguiente dirección: http://localhost:1234.

## Reinicio del contenedor
En caso de ser necesario, por ejemplo que el contenedor se quede sin memoria, no responda, o se caiga el servidor de jupyter-lab, el adminsitrador puede reiniciar el contenedor, para ello debe ejecutar el siguiente comando en el servidor hep-server2:
```bash
   docker restart jupyroot_user
```
donde `jupyroot_user` es el nombre del contenedor del usuario final.

## Eliminar el contenedor de un usuario final

En caso de que el usuario final no vaya a usar más el contenedor, que ocurra un daño critico en el contenedor, o que el usuario final haya olvidado su contraseña, se debe eliminar el contenedor y crear uno nuevo. Para ello se debe ejecutar el siguiente comando en el servidor hep-server2:

```bash
   docker rm -f jupyroot_user
```
donde `jupyroot_user` es el nombre del contenedor del usuario final.